In [2]:
from tqdm import tqdm
import numpy as np
import heapq as heap
from queue import SimpleQueue
from functools import cache
import matplotlib.pyplot as plt

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

with open("data.txt") as file:
    data = file.read()

data = np.array([list(line) for line in data.splitlines()])
dim = data.shape
start = np.argwhere(data == "S")[0]
data[*start] = "."
steps = 64

uncreachables = [[14,  6],
       [36, 40],
       [38, 40],
       [58, 98],
       [92, 68]]

for n in uncreachables:
    data[*n] = "#"

In [3]:
26501365 // dim[0]

202300

In [4]:
26501365 % dim[0]

65

In [5]:
dim[0]

131

In [6]:
dirs = [(0,1), (0,-1), (1, 0), (-1,0)]

def get_neighbours(y, x, data, walls=True):

    neighs = []
    dim = data.shape
    
    for dir in dirs:
        neighs.append((y + dir[0], x + dir[1]))

    neighs = [n for n in neighs if n[0] >= 0 and n[0] < dim[0]]
    neighs = [n for n in neighs if n[1] >= 0 and n[1] < dim[1]]
    neighs = [n for n in neighs if data[n[0], n[1]] == "."] if walls else neighs

    return set(neighs)

current_steps = [(start[0], start[1])]

visited = set()

@cache
def get_possible_combs(cord, steps):

    if steps == 0:
        visited.add((cord[0], cord[1]))
        return 1
    
    combs = 0

    for neigh in get_neighbours(*cord, data):
        combs += get_possible_combs(neigh, steps - 1)
    
    return combs


get_possible_combs((start[0], start[1]), 64)
len(visited)

3743

In [7]:
def search(data, start, walls=True):

    costs = np.ones(data.shape) * np.inf
    costs[start[0], start[1]] = 0

    visited = set()

    pq = []

    heap.heappush(pq, (0, (start[0], start[1])))

    while pq:

        _, node = heap.heappop(pq)
        visited.add(node)

        for adj_node in get_neighbours(*node, data, walls):
            if adj_node in visited: continue
            newCost = costs[node] + 1

            if costs[adj_node] > newCost:
                costs[adj_node] = newCost
                heap.heappush(pq, (newCost, adj_node))


    return costs

w = []

for start in [(0,0), (0,130), (130, 0), (130, 130), (65,65)]:
    costs = search(data, start, walls=False) 
    walls = data[costs <= 65]
    walls = (walls == "#").sum()

    w.append(walls)

In [8]:
steps = 26501365
factor = (steps * 2 + 1) / dim[0]

full_cubes = ((factor // 2) ** 2) * 2 - (factor - 2)
points_inside = (steps * 2 + 1)**2 / 2 + 0.5

sides = factor - 2

s1 = sides // 2
s2 = sides - s1


all_walls = sum(w) * full_cubes

# top corner
all_walls += w[2] + w[3] + w[4]

# left corner
all_walls += w[1] + w[3] + w[4]

# right corner
all_walls += w[0] + w[4] + w[2]

# bottom corner
all_walls += w[0] + w[1] + w[4]

# top -> right
all_walls += s2 * w[2] + s1 * (sum(w) - w[1])

# right -> bottom
all_walls += s2 * w[0] + s1 * (sum(w) - w[3])

# bottom -> left
all_walls += s2 * w[1] + s1 * (sum(w) - w[2])

# rileftght -> top
all_walls += s2 * w[3] + s1 * (sum(w) - w[0])
 
int((points_inside - all_walls + 104 * factor) / 2)

618261433219147